<a href="https://colab.research.google.com/github/jrgreen7/SYSC4906/blob/master/W2024/SYSC4415W24__A3_Student1_Student2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Assignment 3 - Part 2 🥳


---
TA: Elmira Amooei

Questions?: Rather than email, please ask any questions through the Discussion Board on Brightspace. That way, everyone can benefit from the answer

** Deadline: See Brightspace end date**

---

### Working in teams of 2
Complete Part 2 of Assignment 3 in the same team of two students that you formed in Part 1. A single submission per team is required on Brightspace.

### Objective:
In this assignment, we are going to listen to some jazz and disco!
Yes, you read it right. We are using the GTZAN dataset to create a classifier that can tell what the music genre is! 🎷🎸🎹🎺


### Details:
In this part, you are going to implement two models.
The first model is your own **proposed model** (the exact same one you proposed in Part One). And the second model is the **pretrained model** you selected in Part One.

### Instructions:
**DO NOT** wait to the last minute to complete this assignment as it can take hours hours to do this assignment if you are unfmiliar with Python and Machine Learning libraries.

❗Your solutions must be self-contained in this notebook; no other supplementary material or files will be accepted. As soon as I open your assignment's notebook , I will click "runtime" → "run all". Ensure your notebook works properly without any errors and your results are clearly displayed, as instructed.

**Dataset:**
You are provided with two datasets on Brightspace, to be used in Parts 1 & 2 of Assignment. Download the dataset of your choice (*Note that you must use the dataset that you selected in your proposal in Assignment 3 Part 1*), store it in your Google Drive (you need a gmail account to do this). It should be under your "My Drive" (After uploading it to your Google Drive, check the location by right clicking on the file *-> File Information -> Details -> Location*).  **Do not** put it into any sub-folder as  I won't be able to run your code. It will raise an error and you will lose marks.

**Metrics:**
You must create a **confusion matrix** for each of the models you are implementing. A part of your assessment of this assignment is based on the language you use to *interpret what your confusion matrices are showing* and how you would *compare the performance of the two models* based on that.

**Pytorch:** is what we used for Assignment 2, and we are using it here as well.

**Q&A:**
There are some question boxes in *markdown blocks* marked with (🎧🎸) emojis. Your answers to these question blocks should go on the *markdown blocks* marked with (📻🎹) emojis. Just double-click on the emoji and you can start typing.*Note that I will skip over any text answers that are within the code blocks.*

**Libraries:** You can add any library you might need that is not already imported. *If you are using a library that needs to be installed on Colab, please provide the command for it in the specified code block in Section 1.1 Initializations.*

**Optional steps:** There are some code blocks marked as optional. However, feel free to really show your work and add in as much extra info as you want. This may result in bonus points based on what you are doing.😉
Examples:
 * It is optional to use cross-validation in this assignment, but you will get a bonus point for implementing it. 🎙️

 * It is optional to use a grid search to tune your hyperparameters for your own model, but you will get a bonus point for implementing it. 🎙️


**Runtime tips:** I suggest doing your coding and first making sure everything works fine on a CPU. Then do the actual model training on a GPU. To chage your runtime from CPU to GPU and vice versa, select **Runtime** -> **Change Runtime Type** -> under **Hardware accelerator** click on CPU or T4 GPU.

**Submission:** Submit your Notebook as a *.ipynb* file that adopts this naming convention: ***SYSC4415W24_A3_Student1_Student2.ipynb*** on *Brightspace*. No other submission (e.g., through email) will be accepted. (Example file name: SYSC4415W24_A3_ElmiraAmooei_AnthonyFuller.ipynb)

### Grading:
Considering the fact that Elmira is really generous with marking and she wants to see if you understand the point of data exploration, pretraining, confusion matrix, etc., she is very strict with plagiarism. Therefore, for the text answers, she will not show mercy with **suspicious** cases and they will be sent to the Associate Dean for investigation. However, it is alright if you need to consult with Chat-GPT/online resources to figure out **some parts of the code**. Although, remember that I will see a huge ton of submissions that look identical and you might get unlucky! ☠️

You will be evaluated on the performance your model achieved. Try to tune your model to achieve the best possible results you can. You are asked in a *markdown block* to explain what you attempted and explored. (Your work is not compared with any other teams, this is not a competition.)


# 1. Initializations


Some basic ibraries you need are imported here. Make sure you include whatever library you need in this entire notebook in the code block below.

*Note that you **DO NOT** need to use ```cv2``` or ```PIL``` at all! So, please figure out a simpler and more general way of plotting. They should not be imported or used in this notebook as you will be penalized.*

In case you are using any library that requires installation, please paste the installation command for it here.
Leave the code block below if you are not installing any libraries.

In [ ]:
# Libraries to install - leave this code block blank if this does not apply to you
# Please add a brief comment on why you need the library and what it does


In [ ]:
# Libraries you might need
# General
import os
import zipfile
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# For preproccessing
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# For modeling
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torchsummary

# For metrics
from sklearn.metrics import  accuracy_score
from sklearn.metrics import  precision_score
from sklearn.metrics import  recall_score
from sklearn.metrics import  f1_score
from sklearn.metrics import  classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import  roc_auc_score
from sklearn.metrics import confusion_matrix


This code block will mount google drive.

*It will open a new window to get authorizations to use your Google Drive. Just follow the steps. This is a standard process*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 1.2 Create path

*If you absolutely need to change the paths, please do it on the code block below only **and the code block below only**. Because I will run your notebook and if you have other paths defined anywhere else it will raise errors and you will lose mark.*

In [ ]:
# Path for extractions
zip_file_paths = ['/content/drive/My Drive/GTZAN Genre Collection.zip',
                  '/content/drive/My Drive/GTZAN Genre Collection Spectrograms.zip']

dataset_dirs = ['/content/drive/My Drive/GTZAN Genre Collection',
                '/content/drive/My Drive/GTZAN Genre Collection Spectrograms']

for zip_file_path, dataset_dir in zip(zip_file_paths, dataset_dirs):
    if os.path.exists(zip_file_path):
        print(f"Extracting {zip_file_path} to {dataset_dir}")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(dataset_dir)
    else:
        print(f"Zip file {zip_file_path} does not exist.")


#### 1.3 Load your data and see how it looks

The minimum details you need to provide are:

1.   Show how many categories/classes there are in the dataset using ```print()```,
2.   Show how many samples are present in each category using ```print()```,
2.   Plot the class distributions for the dataset using ```matplotlib```,



In [ ]:
# Get a count of the samples


In [ ]:
# Plot the class distributions


In [ ]:
# Any additional data visualizations or exploratory analysis - This is optional, but highly recommended


# 2. Preprocessing

The minimum you need here is to:
1. Create your DataLoaders,
2. Create train and test splits.

Please do normalization/scaling or any additional proposed preproccessing here (Including anyt preprocessing steps you have included in your proposal.)


In [ ]:
# Any additional preproccessing


In [ ]:
# Split the dataset into train and test
# Hint: This is very similar to assignment 2


In [ ]:
# Any additional preproccessing


# 3. Your proposed model

🎧🎸 Explain the steps you took to tune your model and how each step affected your performance. (This answer can be as detailed as you want, don't keep it short.)

* In case you are using grid search for hyperparamter tuning: Why and how are you choosing the grid limits/steps? Give details.

📻🎹

Feel free to add in more code blocks. Please include some comments on what you are doing within the code.

*Add comments to describe the key concepts of the code and its function. It does not need to be line-by-line. Keep it short.*

In [ ]:
# Define your model


In [ ]:
# Initialize the model, loss function, and optimizer


In [ ]:
#Train


In [ ]:
# Test the model suign one chosen metric

In [ ]:
# Compute and plot the confusion matrix


# 4. Your pretrained model

The library below is a great source that possibly covers your proposed pretrained model selected in your proposal. There is a link provided that covers an example implementation. It's very straightforward and needs minimum tweaks.

[Pytorch pretrained models](https://pytorch.org/vision/stable/models.html)

In [ ]:
# Pretrained models and pretrained weights
import torchvision.models as models


Feel free to adjust the code blocks in this section as you see fit.


**Note that your code needs to run flawlessly. (If you are using any pretrained model that is not covered in the library above.) Therefore, please make sure you have every required installations and libraries imported under section 1.1.**

In [ ]:
# Define the pretrained model


In [ ]:
# Initialize the model, loss function, and optimizer


In [ ]:
#Train


In [ ]:
# Test the model using the same chosen metric as your own model


In [ ]:
# Compute and plot the confusion matrix


# Comparisons

🎧🎸 How did you compare your proposed model and the pretrained model in terms of performance? Is your model doing a better/worse job than the pretrained model? Why?
Describing confusion matrices is neccessary to answer this question.
You can also include other metrics, such as accuracy, train-time, etc.

📻🎹

🎧🎸 What will you do differently? Why?
> In terms of the dataset, preproccessing, modeling, etc.

📻🎹

All done! 🎶💃🕺

## 5. Optional challenge!

You are not graded for this part, but I will provide feedback if you gave it a shot!

Believe me, it is a cool piece 🎺

As your model can classify music genres, can you challenge yourselves to have your model listen to you playing a piece of music (e.g., on Spotify) and classify its genre in real-time?

**Hint 01:** ```pyaudio``` is a library you can use.

**Hint 02:** You need to do the same preproccessing on the live data as you did for your model.

**Hint 03:** You need to put your model into the ```.eval()``` state. Don't forget to save the model weights so that you can do load it using ```model.load_state_dict(torch.load('MyModel_weights.pth'))```

**Hint 04:** Your code block needs to listen to the music you are playing, so maybe you can put it in a ```try: while True:``` and ```except KeyboardInterrupt``` loop.